In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

# Best Practices

Here are some best practices for working with diagram.

## Diagram Factories

When you have several sub agent's that share alot of logic, do not repeat yourself by writing a diagram factory.

For example, if we have a different rag agents pointing to different DBs, with different instructions we could write it like so:

```python

# define the factory
def rag_factory(db_path,custom_instructions=None,**db_params):

    # parametrize the document retriever
    document_retriever = DocumentRetriever(db_path,**db_params)
    
    # add your custom instructions to a basic prompt
    prompt = ... 

    with Define(f'RAG',type='decision') as D:
        ... # define the diagram here

    return D

# than we can make different diagrams from the same factory
RAG1 = rag_factory(db_path="...",custom_instructions="...",**db_params)
RAG2 = rag_factory(db_path="...",custom_instructions="...",**db_params)

# and use them normally
r1 = RAG1()
for trace in r1.run({'question':'...'}):
    trace.pprint()

r2 = RAG2()
for trace in r2.run({'question':'...'}):
    trace.pprint()
```

You can see a real example of this in the Interactive QA example.

## Managing contexts for diagram nodes

Often, we have objects that we would like to access through a context manager, ie, accessing a DB's session.

For example, imagine that we have a DB session that is invoked as follows:
```python
with db.get_session(auth_keys) as Session:
    session.execute_query(q)
```

Our problem is that we do not want to instantiate the session when defining the diagram, only when running it.
Meaning we would like something like:


```python

def query1(session):
  session.execute_query('....')

with Define(...) as D:
  ...
  V('q1',query1)
  ...

d=D()

with db.get_session(auth_keys) as Session:
  for trace in d.run(input):
    # somehow the diagram uses the Session we defined
    pass

```

### Bad solution


We could feed the session as input or as State, and rewrite our diagram to get the state

```python
with db.get_session(auth_keys) as Session:
  for trace in d.run(input,state={'Session':Session}):
    pass
```


This is bad for several reasons:
* The session object is not data and will likely not be serializable.
* We are complicating our workflow graph for technical reasons.

### Solution #1  - using factories

```python

def diagram_factory(Session):
    with Define('RAG',type='decision') as D:
        ...
    return D

with db.get_session(auth_keys) as Session:
    d = diagram_factory(Session)
    for trace in d.run(input):
        pass
    
```

This is very straight forward and is a decent solution, though it may not be ideal since it couples defining and running the diagram.

## Solution #2 - create context dependant nodes with your own context managers

> Note, the following has nothing to do with Ladle, it is simply an ordinary design pattern

We can create functions that require an active context to run. There are many ways to do this, but one of them is with the singleton pattern.
We will show an example using the [singleton-decorator](https://pypi.org/project/singleton-decorator/) library.

```python
from singleton_decorator import singleton

@singleton
class DBSession:
    def __init__(self,auth_keys):
        self.active_session= None

    def __enter__(self,auth_keys):
        self.active_session = db.get_session.__enter__(auth_keys)
        return active_session
        
    def __exit__(self):
        self.active_session.__exit__()
        self.active_session = None

    # def you can make specific queries here
    def query_1(self,**params):
        q = '...'.format(**params)
        return self.active_session.execute_query(q)

    # or make a factory for 
    def make_query(self,q_string):

        def run_query(**params)
            q = '...'.format(**params)
            return self.active_session.execute_query(q)
        return run_query
```

Then you can use this as follows:

```python
# when defining diagrams:
with Define('...') as D:
    ...
    V('q1',DBsession().make_query('...'))
    ...

# when running diagrams:
d=D()
with DBSession()(auth_keys) as Session:
    # now the query_function will use the same session inside the diagram
    for trace in d.run(input):
        trace.pprint()
        # we can also add sideeffect
        Session.execute_query('... some metric logging query')

```